<h1> Multivariate Normal Mixture Model

In [1]:
pdf_normal(n,v,μ,Σ) = (2*π)^(-0.5*n) * det(Σ)^(-0.5) * exp(-0.5*(v-μ)'*inv(Σ)*(v-μ))

function EM_normal(t, m,μ,Σ)
    P = size(t,1)
    N = size(t,2)
    G = length(Σ)
    τ = zeros(N,G)
    for g=1:G, n=1:N; τ[n,g] = m[g]*pdf_normal(P,t[:,n],μ[:,g],Σ[g]); end
    τ = τ./sum(τ,2)
    
    T1 = sum(τ,1)
    T2 = t*τ
    T3 = [sum([τ[n,g]*t[:,n]*t[:,n]' for n=1:N]) for g=1:G]
    
    m = T1./N
    μ = T2./T1
    Σ = [(T3[g] - 1/T1[g] * T2[:,g]*T2[:,g]')/T1[g] for g=1:G]
    return m,μ,Σ
end

function some_data(N)
    #some 2 dimensional data in 4 clusters
    n=Int(N/4)
    t1=[1 0.5; 0.5 1]*randn(2,n)*0.4.+[-1,-1]
    t2=[1 -0.5; -0.5 1]*randn(2,n)*0.4.+[-1,+1]
    t3=[1 0.25; 0.25 1]*randn(2,n)*0.4.+[+1,-1]
    t4=[1 -0.25; -0.25 1]*randn(2,n)*0.4.+[+0.5,+0.5]
    t=hcat(t1,t2,t3,t4)
    return t
end

using Plots; gr(xlim=(-2,2), ylim=(-2,2), size=(600,600))

function plot_2d_data_and_components(v,m,μ,Σ)
    a=collect(0:0.1:2.1*π)
    circle=vcat(cos.(a)',sin.(a)')
    covplot=plot()
    scatter!(covplot, v[1,:],v[2,:], title=string("proportions: ",round.(m,2)), lab="data", color=:black)
    pΣ = deepcopy(Σ) #plot with σ size instead of σ² size
    for i=1:length(Σ) #component
        pΣ[i][1,1]=sqrt(pΣ[i][1,1])
        pΣ[i][2,2]=sqrt(pΣ[i][2,2])
        cov=pΣ[i]*circle .+ μ[:,i]
        plot!(covplot,cov[1,:],cov[2,:], lab=string("c",i), width=2)
    end
    plot(covplot)
end

plot_2d_data_and_components (generic function with 1 method)

<h2> Visualize learning the mixture component parameters

In [2]:
using Plots; gr(xlim=(-2,2), ylim=(-2,2), size=(600,600))

P=2 #dimensionality of data
t = some_data(4*50)

G=4 #nr of components
m=ones(G)/G
μ=randn(P,G)
Σ=[eye(P) for g=1:G]

@gif for ITER=1:100
    m,μ,Σ = EM_normal(t, m,μ,Σ)
    plot_2d_data_and_components(t,m,μ,Σ)
end

INFO: Saved animation to /home/andy/dev/githubnotebooks/MixtureModeling/Multivariate_Gaussians/tmp.gif


Plots.AnimatedGif("/home/andy/dev/githubnotebooks/MixtureModeling/Multivariate_Gaussians/tmp.gif")

// Anders